# Load Files

In [135]:
import re
import pandas as pd
import os
import seaborn as sns
import numpy as np
import warnings
import math
warnings.filterwarnings('ignore')

In [136]:
house_train = pd.read_csv('Datasets/train_Wenchang.csv')
house_test = pd.read_csv('Datasets/test_Wenchang.csv')

# Pre Processing

In [137]:
# Combining both house_train and house_test to clean data
house_train # 80 columns
house_test # 79 columns. need SalePrice placeholder
house_test['SalePrice'] = np.nan

# house_test starts on index row 1460 
house_full = pd.concat([house_train, house_test], axis = 0)

# Feature Engineering


### Bath Capacity

In [138]:
# Total Number of bathrooms
house_full['TotBath'] = house_full.FullBath + .5*house_full.HalfBath

# Replace zeros with median
house_full.TotBath.replace(0, house_full.TotBath.median(), inplace=True)

# Replace zero bedrooms with median bedroom #. This is so we can engineer a feature
house_full.BedroomAbvGr.replace(0, house_full.BedroomAbvGr.median(), inplace=True)
house_full.BedroomAbvGr.unique()

# Bath Capacity. The higher the ratio the better
house_full['Bath_Capacity'] = house_full.TotBath / house_full.BedroomAbvGr
house_full.Bath_Capacity.unique()

array([ 0.83333333,  0.66666667,  0.33333333,  0.625     ,  1.5       ,
        1.        ,  0.5       ,  0.75      ,  0.875     ,  0.375     ,
        0.16666667,  1.16666667,  1.25      ,  0.6       ,  0.25      ,
        2.        ,  0.2       ,  0.4       ,  0.7       ,  0.41666667,
        1.33333333,  0.3       ])

### Parking Capacity

In [139]:
# Parking Capacity. The higher the better
house_full['Parking_Capacity'] = house_full.GarageCars / house_full.BedroomAbvGr
house_full.Parking_Capacity.unique()

array([ 0.66666667,  1.        ,  0.75      ,  2.        ,  0.5       ,
        0.33333333,  0.        ,  0.25      ,  0.6       ,  1.5       ,
        3.        ,  0.4       ,  0.16666667,  0.2       ,  1.33333333,
        2.5       ])

### Split Data back into Training/Testing

In [140]:
# split full data back to training and test before imputation
house_train = house_full.iloc[0:1460,:]
house_test = house_full.iloc[1460:,]

### Check for Null in Training

In [141]:
# Check for  null values in training data
house_train.isnull().any().sum() # none

0

### Check for Null in Test

In [142]:
# Check for  null values in training data
house_test.isnull().any().sum() # Only SalePrice is still Null

1

### Finishing touch, delete index column

In [143]:
del house_train['Unnamed: 0']
del house_test['Unnamed: 0']

In [145]:
house_train.SalePrice.head()

0    208500.0
1    181500.0
2    223500.0
3    140000.0
4    250000.0
Name: SalePrice, dtype: float64

In [146]:
house_test.SalePrice.head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: SalePrice, dtype: float64

# Normalize SalePrice variable

In [147]:
sns.distplot(house_train.SalePrice, bins = 20)
sns.distplot([np.log(house_train.SalePrice + 1)], bins = 20)

# Converting SalePrice to Log
house_train.SalePrice = np.log(house_train.SalePrice)


# Dummify and Seperate Training Set to Another Training and Test Set

In [149]:
from copy import deepcopy
import sklearn.model_selection as ms

# Creating a deep copy of training set 
house_train_copy = deepcopy(house_train)

house_train_x = pd.get_dummies(house_train_copy.drop('SalePrice', axis = 1))
house_train_y = house_train_copy.SalePrice.values.reshape(-1,1)

# Create sperate out original training set to the new training and test set
x_train, x_test, y_train, y_test = ms.train_test_split(house_train_x, \
                                                       house_train_y, \
                                                       test_size = 1/5, \
                                                       random_state = 0)


# Fitting Decision Tree

In [150]:
from sklearn import tree

# Train decision tree
tree_model_1 = tree.DecisionTreeRegressor(max_depth = 5)
tree_model_1.fit(x_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

### First try, got 84% R sq on training, and 75% R sq on test

In [151]:
print('Training Score:',tree_model_1.score(x_train, y_train))
print('Test Score:',tree_model_1.score(x_test, y_test))

Training Score: 0.84491034733
Test Score: 0.751848382255


### Error : Training RMSE = .1582 , Testing RMSE = .1938

In [153]:
print('Training RMSE:' , math.sqrt(mean_squared_error(y_train, tree_model_1.predict(x_train))))
print('Test RMSE:' , math.sqrt(mean_squared_error(y_test, tree_model_1.predict(x_test))))

Training RMSE: 0.1582443670260773
Test RMSE: 0.19383012069925434


### These were the top 10 variables

Engineered features not showing up

In [154]:
result = pd.concat([ pd.DataFrame(x_train.columns.values),  pd.DataFrame(tree_model.feature_importances_)], axis = 1)
result.columns = ['feature', 'importance']
result.sort_values(by = 'importance', ascending = False).head(10)

,feature,importance
23,OverallQual,0.611487
12,GarageCars,0.092274
13,GrLivArea,0.077145
26,TotalBsmtSF,0.055339
0,1stFlrSF,0.020065
28,YearBuilt,0.018199
22,OverallCond,0.018186
3,BsmtFinSF1,0.018098
215,MSZoning_RM,0.017328
16,LotArea,0.010954


# Tuning Decision Tree

In [155]:
from sklearn.grid_search import GridSearchCV

tree_model = tree.DecisionTreeRegressor()

param_grid = [{'max_depth': np.arange(1,21), 
              'min_samples_leaf': np.arange(1,10)}]

tree_model.set_params(random_state = 0)

grid_search = GridSearchCV(tree_model, param_grid=param_grid, cv = 10, verbose = 2)

grid_search.fit(x_train, y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits
[CV] max_depth=1, min_samples_leaf=1 .................................
[CV] ........................ max_depth=1, min_samples_leaf=1 -   0.0s
[CV] max_depth=1, min_samples_leaf=1 .................................
[CV] ........................ max_depth=1, min_samples_leaf=1 -   0.0s
[CV] max_depth=1, min_samples_leaf=1 .................................
[CV] ........................ max_depth=1, min_samples_leaf=1 -   0.0s
[CV] max_depth=1, min_samples_leaf=1 .................................
[CV] ........................ max_depth=1, min_samples_leaf=1 -   0.0s
[CV] max_depth=1, min_samples_leaf=1 .................................
[CV] ........................ max_depth=1, min_samples_leaf=1 -   0.0s
[CV] max_depth=1, min_samples_leaf=1 .................................
[CV] ........................ max_depth=1, min_samples_leaf=1 -   0.0s
[CV] max_depth=1, min_samples_leaf=1 .................................
[CV] .......

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s



[CV] max_depth=1, min_samples_leaf=2 .................................
[CV] ........................ max_depth=1, min_samples_leaf=2 -   0.0s
[CV] max_depth=1, min_samples_leaf=2 .................................
[CV] ........................ max_depth=1, min_samples_leaf=2 -   0.0s
[CV] max_depth=1, min_samples_leaf=2 .................................
[CV] ........................ max_depth=1, min_samples_leaf=2 -   0.0s
[CV] max_depth=1, min_samples_leaf=2 .................................
[CV] ........................ max_depth=1, min_samples_leaf=2 -   0.0s
[CV] max_depth=1, min_samples_leaf=2 .................................
[CV] ........................ max_depth=1, min_samples_leaf=2 -   0.0s
[CV] max_depth=1, min_samples_leaf=2 .................................
[CV] ........................ max_depth=1, min_samples_leaf=2 -   0.0s
[CV] max_depth=1, min_samples_leaf=2 .................................
[CV] ........................ max_depth=1, min_samples_leaf=2 -   0.0s
[CV] 

[CV] ........................ max_depth=1, min_samples_leaf=8 -   0.0s
[CV] max_depth=1, min_samples_leaf=8 .................................
[CV] ........................ max_depth=1, min_samples_leaf=8 -   0.0s
[CV] max_depth=1, min_samples_leaf=8 .................................
[CV] ........................ max_depth=1, min_samples_leaf=8 -   0.0s
[CV] max_depth=1, min_samples_leaf=8 .................................
[CV] ........................ max_depth=1, min_samples_leaf=8 -   0.0s
[CV] max_depth=1, min_samples_leaf=8 .................................
[CV] ........................ max_depth=1, min_samples_leaf=8 -   0.0s
[CV] max_depth=1, min_samples_leaf=8 .................................
[CV] ........................ max_depth=1, min_samples_leaf=8 -   0.0s
[CV] max_depth=1, min_samples_leaf=8 .................................
[CV] ........................ max_depth=1, min_samples_leaf=8 -   0.0s
[CV] max_depth=1, min_samples_leaf=8 .................................
[CV] .

[CV] ........................ max_depth=2, min_samples_leaf=5 -   0.0s
[CV] max_depth=2, min_samples_leaf=5 .................................
[CV] ........................ max_depth=2, min_samples_leaf=5 -   0.0s
[CV] max_depth=2, min_samples_leaf=5 .................................
[CV] ........................ max_depth=2, min_samples_leaf=5 -   0.0s
[CV] max_depth=2, min_samples_leaf=5 .................................
[CV] ........................ max_depth=2, min_samples_leaf=5 -   0.0s
[CV] max_depth=2, min_samples_leaf=5 .................................
[CV] ........................ max_depth=2, min_samples_leaf=5 -   0.0s
[CV] max_depth=2, min_samples_leaf=5 .................................
[CV] ........................ max_depth=2, min_samples_leaf=5 -   0.0s
[CV] max_depth=2, min_samples_leaf=5 .................................
[CV] ........................ max_depth=2, min_samples_leaf=5 -   0.0s
[CV] max_depth=2, min_samples_leaf=5 .................................
[CV] .

...

[CV] ....................... max_depth=19, min_samples_leaf=3 -   0.0s
[CV] max_depth=19, min_samples_leaf=3 ................................
[CV] ....................... max_depth=19, min_samples_leaf=3 -   0.0s
[CV] max_depth=19, min_samples_leaf=3 ................................
[CV] ....................... max_depth=19, min_samples_leaf=3 -   0.0s
[CV] max_depth=19, min_samples_leaf=3 ................................
[CV] ....................... max_depth=19, min_samples_leaf=3 -   0.0s
[CV] max_depth=19, min_samples_leaf=3 ................................
[CV] ....................... max_depth=19, min_samples_leaf=3 -   0.0s
[CV] max_depth=19, min_samples_leaf=3 ................................
[CV] ....................... max_depth=19, min_samples_leaf=3 -   0.0s
[CV] max_depth=19, min_samples_leaf=3 ................................
[CV] ....................... max_depth=19, min_samples_leaf=3 -   0.0s
[CV] max_depth=19, min_samples_leaf=3 ................................
[CV] .

[CV] ....................... max_depth=19, min_samples_leaf=8 -   0.0s
[CV] max_depth=19, min_samples_leaf=9 ................................
[CV] ....................... max_depth=19, min_samples_leaf=9 -   0.0s
[CV] max_depth=19, min_samples_leaf=9 ................................
[CV] ....................... max_depth=19, min_samples_leaf=9 -   0.0s
[CV] max_depth=19, min_samples_leaf=9 ................................
[CV] ....................... max_depth=19, min_samples_leaf=9 -   0.0s
[CV] max_depth=19, min_samples_leaf=9 ................................
[CV] ....................... max_depth=19, min_samples_leaf=9 -   0.0s
[CV] max_depth=19, min_samples_leaf=9 ................................
[CV] ....................... max_depth=19, min_samples_leaf=9 -   0.0s
[CV] max_depth=19, min_samples_leaf=9 ................................
[CV] ....................... max_depth=19, min_samples_leaf=9 -   0.0s
[CV] max_depth=19, min_samples_leaf=9 ................................
[CV] .

[CV] ....................... max_depth=20, min_samples_leaf=6 -   0.0s
[CV] max_depth=20, min_samples_leaf=6 ................................
[CV] ....................... max_depth=20, min_samples_leaf=6 -   0.0s
[CV] max_depth=20, min_samples_leaf=6 ................................
[CV] ....................... max_depth=20, min_samples_leaf=6 -   0.0s
[CV] max_depth=20, min_samples_leaf=6 ................................
[CV] ....................... max_depth=20, min_samples_leaf=6 -   0.0s
[CV] max_depth=20, min_samples_leaf=6 ................................
[CV] ....................... max_depth=20, min_samples_leaf=6 -   0.0s
[CV] max_depth=20, min_samples_leaf=6 ................................
[CV] ....................... max_depth=20, min_samples_leaf=6 -   0.0s
[CV] max_depth=20, min_samples_leaf=6 ................................
[CV] ....................... max_depth=20, min_samples_leaf=6 -   0.0s
[CV] max_depth=20, min_samples_leaf=6 ................................
[CV] .

[Parallel(n_jobs=1)]: Done 1800 out of 1800 | elapsed:  1.3min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20]), 'min_samples_leaf': array([1, 2, 3, 4, 5, 6, 7, 8, 9])}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=2)

In [156]:
grid_search.best_params_

{'max_depth': 11, 'min_samples_leaf': 8}

Best hyperparamters are max depth = 11, min_samples_leaf = 8

### Refit Decision Tree with Best Parameters

In [157]:
tree_model = tree.DecisionTreeRegressor(max_depth = 11, 
                                        min_samples_leaf = 8)

tree_model.fit(x_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=11, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=8,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

### R sq is now 91% on training and 77% on test

Though training score increased alot while Test score improves slightly from 75% to 77%

In [159]:
print('Training Score:',tree_model.score(x_train, y_train))
print('Test Score:',tree_model.score(x_test, y_test))

Training Score: 0.911885038298
Test Score: 0.772905474593


### Error : Training RMSE = .1192 , Testing RMSE = .1854

Both Training and Test RMSE lowered in tuned tree. Test RMSE went from .1938 to .1854

In [161]:
print('Training RMSE:' , math.sqrt(mean_squared_error(y_train, tree_model.predict(x_train))))
print('Test RMSE:' , math.sqrt(mean_squared_error(y_test, tree_model.predict(x_test))))

Training RMSE: 0.1192783646765266
Test RMSE: 0.1854240416984672


### Top 10 Important Variables for this fitted tree

Both of the bath engineerd features continue to rank higher in importance compared to the original

In [162]:
result = pd.concat([pd.DataFrame(x_train.columns.values),pd.DataFrame(tree_model.feature_importances_)], axis = 1)
result.columns = ['feature','importance']
result.sort_values(by = 'importance', ascending = False).head(10)


,feature,importance
23,OverallQual,0.607759
13,GrLivArea,0.133346
26,TotalBsmtSF,0.082345
3,BsmtFinSF1,0.023896
69,CentralAir_N,0.021704
36,Parking_Capacity,0.020077
22,OverallCond,0.017090
15,KitchenAbvGr,0.012336
0,1stFlrSF,0.009318
25,TotRmsAbvGrd,0.007056


# Thoughts so far...

I think we can do better on the test score...our model is only fit on one set of observations, so there is high variance in our model.

However, if we fit the model on MULTIPLE sets of observations then we can lower this variance by averaging a bunch of weak models together

Lets try attacking with Bagged Trees to decrease the variance!

# Fitting Bagged Trees

In [163]:
from sklearn.ensemble import BaggingRegressor

bag_tree = BaggingRegressor(
        tree.DecisionTreeRegressor(),
        n_estimators = 500, 
        max_samples = round(len(x_train)*.67), # bag two thirds of training set
        bootstrap = True,      
        random_state = 0,
        oob_score = True
        )

bag_tree.fit(x_train, y_train)

BaggingRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=783, n_estimators=500, n_jobs=1, oob_score=True,
         random_state=0, verbose=0, warm_start=False)

### R sq for Training is 97%, OOB 87% and Test 87%. Test score is close to our OOB score.

Bagged Tree model gave us a much higher Test score, goign from 77% to 87% R sqaured!

In [164]:
print('Training Score:', bag_tree.score(x_train, y_train))
print('OOB Score:', bag_tree.oob_score_)
print('Test Score:', bag_tree.score(x_test, y_test))

Training Score: 0.96564487431
OOB Score: 0.869147517835
Test Score: 0.871577103208


### Error Rate: Training RMSE =  .0745 , Test RMSE = .1394

Test RMSE in Bagged Tree continues to go down .1854 to .1394 from the last model (Tuned Decision Tree)

In [165]:
print('Training RMSE:' , math.sqrt(mean_squared_error(y_train, bag_tree.predict(x_train))))
print('Test RMSE:' , math.sqrt(mean_squared_error(y_test, bag_tree.predict(x_test))))

Training RMSE: 0.07447879108934419
Test RMSE: 0.13943889797618808


## Thoughts so far:

Much improved training and test scores!
But, perhaps these trees are WAY too correlated, still increasing too much variance in our model.
Same predictors will be chosen at the top of all our trees
Lets try attacking Random Forest to decorrelate! Go!

# Fitting Random Forest: First Attempt

In [166]:
from sklearn.ensemble import RandomForestRegressor
import math

rf_tree_1 = RandomForestRegressor(n_estimators = 500,
                                max_features = round(math.sqrt(len(x_train.columns))),
                                bootstrap = True,
                                oob_score = True,
                                random_state = 0)

rf_tree_1.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=17, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=1, oob_score=True, random_state=0,
           verbose=0, warm_start=False)

## R sq for Training: 98%, OOB: 87%, Test: 87%. No improvement in Test Score...

Training score went up 2 points 

In [167]:
print('Training Score:' ,rf_tree_1.score(x_train, y_train))
print('OOB Score:', rf_tree_1.oob_score_)
print('Test Score:' ,rf_tree_1.score(x_test, y_test))

Training Score: 0.982089020291
OOB Score: 0.867143227395
Test Score: 0.871056589436


### Error Rate: Training RMSE =  .0538 , Test RMSE = .1397

Training RMSE lowers by .02 but No change in Test RMSE

In [169]:
print('Training RMSE:' , math.sqrt(mean_squared_error(y_train, rf_tree_1.predict(x_train))))
print('Test RMSE:' , math.sqrt(mean_squared_error(y_test, rf_tree_1.predict(x_test))))

Training RMSE: 0.05377699352930885
Test RMSE: 0.13972119370939215


GRLiveArea now makes it to the top of the list after using Random Forest. The Bath Engineered Features make it to the top 10 important variables as well

In [170]:
result = pd.concat([pd.DataFrame(x_train.columns.values),pd.DataFrame(rf_tree_1.feature_importances_)], axis = 1)
result.columns = ['feature','importance']
result.sort_values(by = 'importance', ascending = False).head(10)

,feature,importance
13,GrLivArea,0.073298
23,OverallQual,0.072784
11,GarageArea,0.042689
26,TotalBsmtSF,0.042659
28,YearBuilt,0.037257
0,1stFlrSF,0.036125
12,GarageCars,0.035706
34,TotBath,0.035510
35,Bath_Capacity,0.033869
10,FullBath,0.033138


# Grid Search CV to Tune Random Forest

In [175]:
from sklearn.grid_search import GridSearchCV

rf_tree_cv = RandomForestRegressor(
                                bootstrap = True,
                                oob_score = True,
                                random_state = 0)

rf_param_grid = [{
        'n_estimators' : [300, 400, 500, 600], #Test[100,250,500,750,1000]
        'max_features' : np.arange(35,38), # First try: np.arange(17,40)
        #'min_samples_leaf' : np.arange(2,9),
        #'max_depth' : np.arange(5,11)
        }]

rf_grid_search = GridSearchCV(rf_tree_cv, param_grid = rf_param_grid, cv = 5)

rf_grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=True, random_state=0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [300, 400, 500, 600], 'max_features': array([35, 36, 37])}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

## Best parameters: Max_features = 36, n_estimators = 500

In [177]:
rf_grid_search.best_params_

{'max_features': 36, 'n_estimators': 500}

# Refit Random Forest with Tuned Hyperparameters

In [178]:
rf_tree = RandomForestRegressor(n_estimators = 500,
                                max_features = 36,
                                bootstrap = True,
                                oob_score = True,
                                random_state = 0)


rf_tree.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=36, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=1, oob_score=True, random_state=0,
           verbose=0, warm_start=False)

## R sq for Training: 98%, OOB: 87%, Test: 88%

Test score was <1% higher in the tuned Random Forest than the untuned Random Forest model

In [179]:
print('Training Score:', rf_tree.score(x_train, y_train))
print('OOB Score:', rf_tree.oob_score_)
print('Test Score:', rf_tree.score(x_test, y_test))

Training Score: 0.982891279992
OOB Score: 0.873416643755
Test Score: 0.877668748922


### Error: Training RMSE = .0526

In [180]:
from sklearn.metrics import mean_squared_error

print('Training RMSE:' , math.sqrt(mean_squared_error(y_train, rf_tree.predict(x_train))))

Training RMSE: 0.05255882012222655


### Error: Test RMSE = .1361

In [181]:
print('Test RMSE:', math.sqrt(mean_squared_error(y_test, rf_tree.predict(x_test))))

Test RMSE: 0.13609163115664372


### Top 10 Important Variables

In [183]:
result = pd.concat([pd.DataFrame(x_train.columns.values),pd.DataFrame(rf_tree.feature_importances_)], axis = 1)
result.columns = ['feature','importance']
result.sort_values(by = 'importance', ascending = False).head(10)

,feature,importance
23,OverallQual,0.100629
13,GrLivArea,0.097593
12,GarageCars,0.053279
11,GarageArea,0.051931
28,YearBuilt,0.049150
26,TotalBsmtSF,0.047400
10,FullBath,0.042848
34,TotBath,0.040406
0,1stFlrSF,0.037287
35,Bath_Capacity,0.036373


In [198]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(6, 15)

# Plot top ten features
sns.set_color_codes("pastel")
sns.barplot(x="importance", y="feature", data=result,
            label="Total", color="b")


SyntaxError: invalid syntax (<ipython-input-198-4151baa393d2>, line 10)

## Thoughts so far:

The random forest models had better training and test scores as well as lower MSE than the decision tree. 

The tuned Random Forest model had slightly higher R^2 than the untuned Random Forest, BUT it also had a slightly higher 